# Setup
This section contains all codes to setup the notebook.

In [ ]:
# Reload all modules at every code execution
%load_ext autoreload
%autoreload 2

In [ ]:
# Dataloader

# Graph1 (Sven) - Growth by Age Rating
Shows how TV-MA and R-related content exploted post-2015

# Graph2 (Sven) - Composition by Rating
Confirms the shift: mature content now dominates share.


# Graph3 (Sven) Genre vs Rating
Reveals that Drama, Thriller, HOrror = mature, Kids, Family = G/PG

# Graph4 (Boris) - Top Countries by Mature Titles
Which regions are leading to shift to TV-MA?

# Graph5 (Boris) - Runtimes by Age Rating
Mature titles tend to be longer, especially TV shows.

# Graph6 (Elias) - Genre Popularity by Country
What genres are dominant in U.S., India, UK, Japan?